In [1]:
import hashlib
import random

In [2]:
# read in the data
file1 = open("S1.txt","r")
s1 = file1.read()
file1.close()

file1 = open("S2.txt","r")
s2 = file1.read()
file1.close()

In [3]:
def fixedDomainHash(item, salt, domainSize):
    o = item + str(salt)
    o = str.encode(o)
    h = hashlib.sha224(o)
    d = h.hexdigest()
    d_int = int(d, 16) % domainSize
    return d_int

In [4]:
class countMatrix:
    
    def __init__(self, inputString, k, hashCount):
        # initialize values which will be referenced repeatedly
        self.k = k
        self.saltList = random.sample(range(1, 1000), hashCount)
        self.cm = [[0 for i in range(k)] for j in range(len(self.saltList))]
        self.elementSet = set()

        # build the data count matrix data structure (input step)
        for i in range(len(inputString)):
            for j in range(len(self.saltList)):
                if inputString[i] not in self.elementSet:
                    self.elementSet.add(inputString[i])
                kIndex = self.fixedDomainHash(inputString[i], self.saltList[j], k)
                self.cm[j][kIndex] += 1
                
    def fixedDomainHash(self, item, salt, domainSize):
        o = item + str(salt)
        o = str.encode(o)
        h = hashlib.sha224(o)
        d = h.hexdigest()
        d_int = int(d, 16) % domainSize
        return d_int
        
    # query functionality of Count-Min Sketch Algorithm
    def query(self, element):
        minResult=float('inf')
        for i in range(len(self.saltList)):
            kIndex = self.fixedDomainHash(element, self.saltList[i], self.k)
            if self.cm[i][kIndex] <= minResult:
                minResult = self.cm[i][kIndex]
        return minResult
    
    # functionality to get all observed approximations
    def getAll(self):
        resultDic = {}
        for element in self.elementSet:
            resultDic[element] = self.query(element)
        return resultDic

In [5]:
s1Ds = countMatrix(s1, 10, 5)

In [6]:
s2Ds = countMatrix(s2, 10, 5)

In [7]:
# now for the assignment specific questions
charSet = ['a', 'b', 'c']

In [8]:
for char in charSet:
    q1 = s1Ds.query(char)
    print("Character: {}".format(char))
    print("Approximated Count in S1: {}".format(q1))
    print("Approximated Frequency in S1: {}".format(q1 / len(s1)))
    print("")

Character: a
Approximated Count in S1: 959642
Approximated Frequency in S1: 0.31988066666666665

Character: b
Approximated Count in S1: 659314
Approximated Frequency in S1: 0.21977133333333335

Character: c
Approximated Count in S1: 480804
Approximated Frequency in S1: 0.160268



In [9]:
for char in charSet:
    q2 = s2Ds.query(char)
    print("Character: {}".format(char))
    print("Approximated Count in S2: {}".format(q2))
    print("Approximated Frequency in S2: {}".format(q2 / len(s2)))
    print("")

Character: a
Approximated Count in S2: 2040477
Approximated Frequency in S2: 0.51011925

Character: b
Approximated Count in S2: 799400
Approximated Frequency in S2: 0.19985

Character: c
Approximated Count in S2: 440444
Approximated Frequency in S2: 0.110111



In [10]:
e1 = len(s1) / 10
e2 = len(s2) / 10
print("Maximum Error for s1: {}".format(e1))
print("Maximum Error for s2: {}".format(e2))

s1AllCounts = s1Ds.getAll()
s2AllCounts = s2Ds.getAll()

Maximum Error for s1: 300000.0
Maximum Error for s2: 400000.0


In [11]:
def printCutOffResults(dic, error, dataLength):
    mustList = []
    maybeList = []
    for key in dic:
        if (dic[key] - error) > (dataLength / 5):
            mustList.append(key)
        elif (dic[key] - error) < (dataLength / 5) and (dic[key] - error) > (dataLength / 10):
            maybeList.append(key)
    return mustList, maybeList

In [12]:
s1MustList, s1MaybeList = printCutOffResults(s1AllCounts, e1, len(s1))

In [13]:
s2MustList, s2MaybeList = printCutOffResults(s2AllCounts, e2, len(s2))

In [14]:
print("Count-Min Sketch on s1 with k=10, t=5")
print("")
print("Elements in s1 that must be greater than 20% frequency")
print(s1MustList)
print("")
print("Elements in s1 that might be greater than 20% frequency")
print(s1MaybeList)
print("")

print("Count-Min Sketch on s2 with k=10, t=5")
print("")
print("Elements in s2 that must be greater than 20% frequency")
print(s2MustList)
print("")
print("Elements in s2 that might be greater than 20% frequency")
print(s2MaybeList)

Count-Min Sketch on s1 with k=10, t=5

Elements in s1 that must be greater than 20% frequency
['a']

Elements in s1 that might be greater than 20% frequency
['b']

Count-Min Sketch on s2 with k=10, t=5

Elements in s2 that must be greater than 20% frequency
['a']

Elements in s2 that might be greater than 20% frequency
[]
